## Deploy model to Azure Container Instance (ACI)

### Setup environment

In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
 
import azureml.core

# display the core SDK version number
print("Azure ML SDK Version: ", azureml.core.VERSION)

/anaconda/envs/py36/lib/python3.6/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.25.2) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


Azure ML SDK Version:  1.0.39


### Retrieve the model

In [2]:
from azureml.core import Workspace
from azureml.core.model import Model
import os 
ws = Workspace.from_config()
model=Model(ws, 'unet_membrane')

model.download(target_dir=os.getcwd(), exist_ok=True)

# verify the downloaded model file
file_path = os.path.join(os.getcwd(), "unet_membrane.hdf5")

os.stat(file_path)

os.stat_result(st_mode=33279, st_ino=20, st_dev=50, st_nlink=1, st_uid=0, st_gid=0, st_size=372551468, st_atime=1558472305, st_mtime=1558472305, st_ctime=1558472305)

### If you are smart you would test your model here

### Create scoring scipt

In [ ]:
%%writefile score.py
import json
import numpy as np
import os
from keras.models import load_model

from azureml.core.model import Model

def init():
    global model
    # retrieve the path to the model file using the model name
    model_path = Model.get_model_path('unet_membrane')
    model = load_model(model_path)
    

def run(raw_data):
    try:
        data = np.array(json.loads(raw_data)['data'])
        # make prediction
        y_hat = model.predict(data)
        # you can return any data type as long as it is JSON-serializable
        return y_hat.tolist()
    except Exception as e:
        error = str(e)
        return error

### Create environment

In [ ]:
from azureml.core.conda_dependencies import CondaDependencies 

myenv = CondaDependencies()
myenv.add_conda_package("keras")

with open("myenv.yml","w") as f:
    f.write(myenv.serialize_to_string())

### Create config

In [5]:
from azureml.core.webservice import AciWebservice

aciconfig = AciWebservice.deploy_configuration(cpu_cores=2, 
                                               memory_gb=5, 
                                               tags={"data": "membrane"}, 
                                               description='Predict membrane boundaries with keras')

### Deploy in ACI

In [6]:
%%time
from azureml.core.webservice import Webservice
from azureml.core.image import ContainerImage

# configure the image
image_config = ContainerImage.image_configuration(execution_script="score.py", 
                                                  runtime="python", 
                                                  conda_file="myenv.yml")

CPU times: user 311 µs, sys: 77 µs, total: 388 µs
Wall time: 304 µs


In [10]:
%%time
#deploy webservice
service = Webservice.deploy_from_model(workspace=ws,
                                       name='unet-membrane-svc',
                                       deployment_config=aciconfig,
                                       models=[model],
                                       image_config=image_config)

service.wait_for_deployment(show_output=True)

print(service.scoring_uri)

Creating image

Image creation operation finished for image unet-membrane-svc:6, operation "Succeeded"
Creating service
Running.................................
SucceededACI service creation operation finished, operation "Succeeded"
http://7205e8e6-7f2b-43d1-abb5-8fa4805dbfac.westus.azurecontainer.io/score
CPU times: user 2.17 s, sys: 183 ms, total: 2.36 s
Wall time: 8min 48s


In [8]:
#Run to kill the deployment
service.delete()

### Test the deployment

In [52]:
import json
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import skimage.transform as trans

fig=plt.figure(figsize=(256, 256))

sample_images = mpimg.imread(os.getcwd()+"/data/membrane/test/1.png")
fig.add_subplot(1, 3, 1)
plt.imshow(sample_images, cmap='gray', vmin=0, vmax=1)


sample_images = trans.resize(sample_images,(256,256))
sample_images = np.expand_dims(sample_images, axis=2)

demo_images = mpimg.imread(os.getcwd()+"/data/membrane/test/1_predict.png")
fig.add_subplot(1, 3, 2)
plt.imshow(demo_images, cmap='gray', vmin=0, vmax=1)

test_samples = json.dumps({"data": [sample_images.tolist()]})
test_samples = bytes(test_samples, encoding='utf8')


# predict using the deployed model
# service takes in an image of shape (1, 256,256,1) as a list in json
result = np.array(service.run(input_data=test_samples))
fig.add_subplot(1, 3, 3)
plt.imshow(result.squeeze(), cmap='gray', vmin=0, vmax=1)

plt.show()

(256, 256, 1)
